In [1]:
!pip install transformers torch accelerate

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or


In [3]:
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

SYSTEM_PROMPT = """
You are a helpful AI agent.
Answer clearly and concisely.
"""

class FastAIAgent:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        self.model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            device_map="auto",
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
        )

    def generate(self, user_input):
        prompt = f"""<system>{SYSTEM_PROMPT}</system>
<user>{user_input}</user>
<assistant>"""

        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        outputs = self.model.generate(
            **inputs,
            max_new_tokens=150,      # 🔥 smaller = faster
            temperature=0.5,
            do_sample=True,
            pad_token_id=self.tokenizer.eos_token_id
        )

        return self.tokenizer.decode(outputs[0], skip_special_tokens=True).split("<assistant>")[-1]


def run_agent():
    agent = FastAIAgent()
    print("⚡ Fast Open-Source AI Agent (TinyLlama). Type 'quit' to exit.\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() in {"quit", "exit"}:
            print("Agent: Goodbye 👋")
            break

        print("Agent:", agent.generate(user_input), "\n")


In [4]:
if __name__ == "__main__":
    run_agent()

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:82: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

⚡ Fast Open-Source AI Agent (TinyLlama). Type 'quit' to exit.

You: Who is Lin Dan?
Agent: Lin Dan is a professional Chinese badminton player who is widely regarded as one of the greatest badminton players of all time. He has won 12 Olympic gold medals, 21 World Championship gold medals, and 36 other international badminton championships. He is also known for his aggressive and powerful game style and his ability to adapt to different playing conditions. 

You: exit
Agent: Goodbye 👋
